In [1]:
 # data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

 # visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

train_df = pd.read_csv('EVM/train_longevity.csv')
test_df = pd.read_csv('EVM/test_longevity.csv')
combine = [train_df, test_df]

train_df.shape      # мы увидим информацию о размерности нашего датафрейма 
train_df.info()     # покажет информацию о размерности данных 
                          # описание индекса, количество not-a-number элементов 
train_df.head()         # показывает первые 10 значений датасета
train_df.describe()     # показывает статистики count,mean, std, min, 25%-50%-75% percentile, max 
train_df.nunique()     # количество уникальных значений для каждого столбца

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         857 non-null    int64 
 1   Longevity  857 non-null    int64 
 2   Education  857 non-null    int64 
 3   Sex        857 non-null    object
 4   Age        680 non-null    object
 5   Pet        857 non-null    int64 
 6   Children   857 non-null    int64 
 7   Region     857 non-null    object
 8   Activity   857 non-null    int64 
 9   MedExam    196 non-null    object
 10  Sport      236 non-null    object
dtypes: int64(6), object(5)
memory usage: 73.8+ KB


Id           857
Longevity      2
Education      3
Sex            2
Age           12
Pet            7
Children       7
Region        51
Activity     229
MedExam        5
Sport          2
dtype: int64

In [2]:
### 1 Признак Age должен быть дополнен и фильтрован для обработки алгоритмами
# Изменить недопустимое значение на idmax
# idmax - возраст с наибольшим количеством

train_df['Age'].unique()
test_df['Age'].unique()

def is_number(a):
    try:
        a = float(a)
    except:
        return False
    
    return not np.isnan(a)

idmax = train_df['Age'].value_counts().idxmax()
train_df['Age'] = train_df['Age'].map(lambda v: v if is_number(v) else idmax).astype(float)
test_df['Age'] = test_df['Age'].map(lambda v: v if is_number(v) else idmax).astype(float)

In [3]:
### 2 Признак MedExam может быть отброшен
train_df = train_df.drop(columns="MedExam")
test_df = test_df.drop(columns="MedExam")
train_df.head()

,Id,Longevity,Education,Sex,Age,Pet,Children,Region,Activity,Sport
0,1,0,3,female,73.0,1,0,21,725,NaN
1,2,1,1,male,75.0,1,0,17,7128,+
2,3,1,3,male,73.0,0,0,31,793,NaN
3,4,1,1,male,74.0,1,0,11,5310,NaN
4,5,0,3,female,74.0,0,0,37,805,NaN


In [4]:
### 3 Необходимо дополнить функцию Sport, поскольку она также может соотноситься с долголетием.
 
# Статус спорта: (чем ниже статус, тем больше спорта в жизни)
# 2 - плохо, 1 - хорошо, 0 - отлично
# зависит от кол-ва шагов(активности) и признака IsAlone
 
# mean() - среднее значение
act_avg = int(train_df["Activity"].mean())
# print(act_avg)
 
def groupAct(act):
    if(act <= act_avg):
        return 1
    else:
        return 0
 
train_df["Sport"] = train_df["Activity"].map(lambda a: groupAct(a))
train_df.head()

,Id,Longevity,Education,Sex,Age,Pet,Children,Region,Activity,Sport
0,1,0,3,female,73.0,1,0,21,725,1
1,2,1,1,male,75.0,1,0,17,7128,0
2,3,1,3,male,73.0,0,0,31,793,1
3,4,1,1,male,74.0,1,0,11,5310,0
4,5,0,3,female,74.0,0,0,37,805,1


In [5]:
### 4 Признак Region может быть исключен из нашего анализа
train_df = train_df.drop(columns="Region")
test_df = test_df.drop(columns="Region")
train_df.head()

,Id,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport
0,1,0,3,female,73.0,1,0,725,1
1,2,1,1,male,75.0,1,0,7128,0
2,3,1,3,male,73.0,0,0,793,1
3,4,1,1,male,74.0,1,0,5310,0
4,5,0,3,female,74.0,0,0,805,1


In [6]:
### 5 Пизнак Id может быть удален из набора обучающих данных
train_df = train_df.drop(columns="Id")

In [7]:
### 6 Cоздать новый признак под названием Family (Семья на основе детей и домашних животных),
#     чтобы получить общее количество членов семьи.
train_df["Family"] = train_df["Pet"] + train_df["Children"]

In [8]:
### 7 Заздать новый признак IsAlone, проживающих одиноко без домашних животных и родственников.
train_df["IsAlone"] = train_df.apply(lambda row: 1 if not row.Family else 0, axis=1)
train_df.head()

,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone
0,0,3,female,73.0,1,0,725,1,1,0
1,1,1,male,75.0,1,0,7128,0,1,0
2,1,3,male,73.0,0,0,793,1,0,1
3,1,1,male,74.0,1,0,5310,0,1,0
4,0,3,female,74.0,0,0,805,1,0,1


In [9]:
### 7.2 Cоздать новый признак для возрастных групп (Age,Education), указывающий на социальный статус пожилого человека

# (...,70](70,72],(72,74],(74,76],(76,78],(80,..)}
# min i = 0, max i = 6
def groupAge(age):
    interval =  np.arange(70, 82, 2)
    l = len(interval)
    i = 0
    while i < l and age > interval[i]:
        i += 1
    return i

train_df["Age*Education"] = train_df["Age"].map(lambda a: groupAge(a)) * train_df["Education"]
train_df.head(10)

,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education
0,0,3,female,73.0,1,0,725,1,1,0,6
1,1,1,male,75.0,1,0,7128,0,1,0,3
2,1,3,male,73.0,0,0,793,1,0,1,6
3,1,1,male,74.0,1,0,5310,0,1,0,2
4,0,3,female,74.0,0,0,805,1,0,1,6
5,0,3,female,73.0,0,0,846,1,0,1,6
6,0,1,female,77.0,0,0,5186,0,0,1,4
7,0,3,female,70.0,3,1,2108,1,4,0,0
8,1,3,male,73.0,0,2,1113,1,2,0,6
9,1,2,male,72.0,1,0,3007,1,1,0,2


In [10]:
### 8  Cоздать новый признак диапазонов возрастного равновесия на основе признака Age, т.к. это поможет разделить пожилых людей 
# на группы условно равновесного состояния (смертность повышается в определенные периоды времени между 70 и 80 годами, 
# и в другие моменты резко снижается) для следующих интервалов: { (...,70](70,72],(72,74],(74,76],(76,78],(80,..)}  
#                                                                                                   ==  (78, 80], (80, ...) ==
# Status: от 0 до 9
# Чем больше статус, тем выше смертность
# зависит от возраста, спорта и статуса IsAlone
 
train_df["Status"] = train_df["Age"].map(lambda a: groupAge(a)) + train_df["Sport"] + train_df["IsAlone"]
train_df.head(10)

,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,Age*Education,Status
0,0,3,female,73.0,1,0,725,1,1,0,6,3
1,1,1,male,75.0,1,0,7128,0,1,0,3,3
2,1,3,male,73.0,0,0,793,1,0,1,6,4
3,1,1,male,74.0,1,0,5310,0,1,0,2,2
4,0,3,female,74.0,0,0,805,1,0,1,6,4
5,0,3,female,73.0,0,0,846,1,0,1,6,4
6,0,1,female,77.0,0,0,5186,0,0,1,4,5
7,0,3,female,70.0,3,1,2108,1,4,0,0,1
8,1,3,male,73.0,0,2,1113,1,2,0,6,3
9,1,2,male,72.0,1,0,3007,1,1,0,2,2
